Bank Account Management System

In [5]:
import pickle # For data serialization and deserialization
import os # For file operations
import random # For generating random account numbers and PINs
import numpy as np # For potential numerical operations
import datetime as dt # For timestamps in transactions

# File paths for storing account and transaction data
data_file = 'accounts_data.pkl' # File to store account details
transactions_file = 'transactions_data.pkl' # File to store transaction history

# Dictionaries to store account and transaction data in memory
account_details = {} # Structure: {pancard: [name, account_type, balance, account_no, password]}
details = [] # Temporary list to hold account details during creation
transactions = {} # Structure: {pancard: {timestamp: [amount, type, balance]}}

# Load existing data from files if they exist
if os.path.exists(data_file):
    with open(data_file, 'rb') as file:
        account_details = pickle.load(file)

if os.path.exists(transactions_file):
    with open(transactions_file, 'rb') as file:
        transactions = pickle.load(file)

# Function to save account and transaction data to files
def save_data():
    with open(data_file, 'wb') as file:
        pickle.dump(account_details, file)
    with open(transactions_file, 'wb') as file:
        pickle.dump(transactions, file)

# Display welcome message
print("*"*50)
text = "Welcome to WsCube Bank"
padded_text = text.center(50)
print(padded_text)
print("*"*50)

# Main program loop
while True:
    # Display menu options
    print("\nMenu Options: ")
    print("-"*13)
    print("""
    Press 1 for Opening an Account
    Press 2 to View Account Details
    Press 3 to withdraw/deposit/transfer
    Press 4 to view transaction history
    Press 5 to exit the program
    """)
    print("-"*50)

    # Get user choice and validate it
    choice = input("Enter your choice between 1-5: ")

    if not choice.isdigit() or int(choice) < 1 or int(choice) > 5:
        print("Invalid input. Please enter a number between 1 and 5.")
        continue

    choice = int(choice)
    print("-"*50)

    # Option 1: Open an Account
    if choice == 1:
        pancard = input("Enter Your Pancard Number    : ")
        name = input("Enter your name here         : ")
        account_type = input("Account Type (Savings/Credit): ")
        initial_deposit = float(input("Enter initial deposit        : "))
        account_no = "9876543210" + str(random.randint(1000, 9999))
        password = str(random.randint(100000, 999999))
    
       # Generate unique account number and password
        while account_no in [details[3] for details in account_details.values()]:
            account_no = "9876543210" + str(random.randint(1000, 9999))
    
        # Store account details
        details = [name, account_type, initial_deposit, account_no, password]
        account_details[pancard] = details  # Store details for this pancard
    
        # Save data to file
        save_data()

        # Display account creation details
        print("-" * 50)
        print(f"Account Created Successfully!")
        print("*" * 32)
        print(f"Account No     : {account_no}")
        print(f"Pin Number     : {password}")
        print(f"Name           : {name}")       
        print(f"Account Type   : {account_type}") 
        print(f"Initial Balance: {initial_deposit}")
        print("*" * 32)

    # Option 2: View Account Details
    elif choice == 2:  
       # Get account number and password from the user
        ask_account_number = input("Enter Account Number   : ")
        ask_password = input("Enter your 6-Digit Pin : ")
    
        # Search for the account
        account_found = None
        for pancard, details in account_details.items():
            if details[3] == ask_account_number and details[4] == ask_password:
                account_found = details
                break
                
        # Display account details if found
        if account_found:            
            print("*" * 50)
            text1 = "Account Details"
            padded_text1 = text1.center(50)
            print(padded_text1)
            print("*" * 50)
            print("Account Holder's Name: ", account_found[0])
            print("Account Type         : ", account_found[1])
            print("Account Number       : ", account_found[3])
            print("Account Balance      : ", account_found[2])
            print("*" * 50)
        else:
            print("Invalid account number or pin number.")
    
    # Option 3: Withdraw/Deposit/Transfer
    elif choice == 3:        
        text2 = "Withdraw - Deposit - Transfer"
        padded_text2 = text2.center(50)
        print(padded_text2)
        print("*"*50)
        
        # Sub-menu for transaction types
        print("""
        Press 1 to withdraw
        Press 2 to deposit
        Press 3 to transfer
        """)
        
        print("-"*50)
    
        # Get transaction choice and validate it
        n_choice = input("Enter your choice between 1-3: ")    
        
        if not n_choice.isdigit() or int(n_choice) < 1 or int(n_choice) > 3:
            print("Enter a valid option.")
            continue    
        
        n_choice = int(n_choice)
        print("-"*50)
                            
        # Handle each transaction type
        if n_choice == 1: # Withdraw
            # Prompt the user to enter their account number and pin
            ask_account_number = input("Enter Account Number              : ")
            ask_password = input("Enter your 6-Digit Pin            : ")
                                
            # Initialize a variable to store the account found status
            account_found = None

            # Loop through the account details to find the matching account number and pin
            for pancard, details in account_details.items():
                if details[3] == ask_account_number and details[4] == ask_password:
                    account_found = pancard  
                    break # Exit the loop once account is found

            # Check if a valid account was found based on the provided details
            if account_found: 
                # Prompt the user to enter the withdrawal amount
                withdraw_amount = float(input("Enter amount you wish to withdraw : "))
        
                # Check if the user has enough balance to make the withdrawal
                if withdraw_amount > account_details[account_found][2]:
                    # Inform the user if the balance is insufficient
                    print("Sorry! Insufficient Balance")
                else:
                    # Deduct the withdrawal amount from the account balance
                    account_details[account_found][2] -= withdraw_amount
                    # Get the current timestamp for transaction record
                    timestamp = dt.datetime.now()
                    ts = timestamp.strftime("%Y-%m-%d %H:%M:%S")
                    # Prepare transaction data (withdrawal amount, transaction type, remaining balance)
                    t_data = [withdraw_amount, "Debited", account_details[account_found][2]]
        
                    # Check if the account already has a transaction record; if not, initialize it
                    if account_found not in transactions:
                        transactions[account_found] = {}

                    # Add the transaction data with the timestamp as key
                    transactions[account_found][ts] = t_data
        
                    # Save updated data (e.g., balance, transaction history) to the system/database
                    save_data()
                    # Provide feedback to the user on successful withdrawal and show remaining balance
                    print("-" * 50)
                    print(f"Withdrawal Successful! Remaining Balance: {account_details[account_found][2]}")
                    print("-" * 50)
                    print("Transaction Details:")
                    print("-" * 33)
                    print(f"Date & Time : {ts}")
                    print(f"Amount      : {withdraw_amount}")
                    print(f"Status      : Debited")
                    print(f"Balance     : {account_details[account_found][2]}")
            else:
                # If no valid account is found, inform the user
                print("Invalid account number or pin.")
        
        elif n_choice == 2: # Deposit
            # Prompt the user to enter their account number and pin
            ask_account_number = input("Enter Account Number              : ")
            ask_password = input("Enter your 6-Digit Pin            : ")
        
            # Initialize a variable to store the account found status
            account_found = None
            # Loop through the account details to find the matching account number and pin
            for pancard, details in account_details.items():
                # Compare the account number and pin with the user's input
                if details[3] == ask_account_number and details[4] == ask_password:
                    # If a match is found, store the PAN card (account identifier) and exit the loop
                    account_found = pancard 
                    break # Exit the loop once account is found
                    
            # Check if a valid account was found based on the provided details
            if account_found: 
                # Prompt the user to enter the deposit amount
                deposit_amount = float(input("Enter amount you wish to deposit  : "))
                                            
                # Add the deposit amount to the current balance of the account
                account_details[account_found][2] += deposit_amount
                # Get the current timestamp for the transaction record
                timestamp = dt.datetime.now()
                ts = timestamp.strftime("%Y-%m-%d %H:%M:%S") # Format timestamp as string
                # Prepare transaction data (deposit amount, transaction type, updated balance)
                t_data = [deposit_amount, "Credited", account_details[account_found][2]]
        
               # Check if the account already has a transaction record; if not, initialize it
                if account_found not in transactions:
                    transactions[account_found] = {}
                # Add the transaction data with the timestamp as key
                transactions[account_found][ts] = t_data
        
                # Save updated data (e.g., balance, transaction history) to the system/database
                save_data()

                # Provide feedback to the user on successful deposit and show new balance
                print("-" * 50)
                print(f"Deposit Successful! New Balance: {account_details[account_found][2]}")
                print("-" * 50)
                print("Transaction Details:")
                print("-" * 33)
                print(f"Date & Time : {ts}")
                print(f"Amount      : {deposit_amount}")
                print(f"Status      : Credited")
                print(f"Balance     : {account_details[account_found][2]}")
            else:
                # If no valid account is found, inform the user
                print("Invalid account number or pin.")
                    
        else: # Transfer
            # Prompt the user to enter their account number and pin
            ask_account_number = input("Enter Your Account Number          : ")
            ask_password = input("Enter Your 6-Digit Pin             : ")
        
            # Initialize a variable to store the sender account found status
            sender_account_found = None
            # Loop through the account details to find the matching sender account number and pin
            for pancard, details in account_details.items():
                # Compare the account number and pin with the user's input
                if details[3] == ask_account_number and details[4] == ask_password:
                    # If a match is found, store the PAN card (account identifier) and exit the loop
                    sender_account_found = pancard 
                    break # Exit the loop once the sender account is found

            # Check if a valid sender account was found based on the provided details
            if sender_account_found: 
                # Prompt the user to enter the recipient's account number and transfer amount
                recipient_account_no = input("Enter Recipient's Account Number  : ")
                transfer_amount = float(input("Enter amount you wish to transfer : "))
        
                # Initialize a variable to store the recipient account found status
                recipient_account_found = None
                # Loop through the account details to find the matching recipient account number
                for pancard, details in account_details.items():
                    # If a match is found, store the PAN card (account identifier) for the recipient
                    if details[3] == recipient_account_no:
                        recipient_account_found = pancard
                        break # Exit the loop once the recipient account is found
                # Check if the recipient account was found
                if recipient_account_found:  
                    # Check if the sender has sufficient balance to make the transfer
                    if transfer_amount > account_details[sender_account_found][2]:
                        # Inform the user if there is insufficient balance
                        print("Sorry! Insufficient Balance")
                    else:
                        # Deduct the transfer amount from the sender's account balance
                        account_details[sender_account_found][2] -= transfer_amount
                        # Add the transfer amount to the recipient's account balance
                        account_details[recipient_account_found][2] += transfer_amount
                        
                        # Get the current timestamp for transaction records
                        timestamp = dt.datetime.now()
                        ts = timestamp.strftime("%Y-%m-%d %H:%M:%S")
                        # Prepare transaction data for the sender (amount, transaction type, updated balance)
                        sender_t_data = [transfer_amount, "Transfer/Debited", account_details[sender_account_found][2]]
                        
                        # Check if the sender already has a transaction record; if not, initialize it
                        if sender_account_found not in transactions:
                            transactions[sender_account_found] = {}
                        # Add the transaction data with the timestamp as key for the sender
                        transactions[sender_account_found][ts] = sender_t_data
        
                        # Prepare transaction data for the recipient (amount, transaction type, updated balance)
                        recipient_t_data = [transfer_amount, "Transfer/Credited", account_details[recipient_account_found][2]]
                        # Check if the recipient already has a transaction record; if not, initialize it
                        if recipient_account_found not in transactions:
                            transactions[recipient_account_found] = {}
                        # Add the transaction data with the timestamp as key for the recipient
                        transactions[recipient_account_found][ts] = recipient_t_data
        
                        # Save updated data (e.g., balance, transaction history) to the system/database
                        save_data()
                        
                        # Provide feedback to the user on successful transfer and show new balance for the sender
                        print("-" * 50)
                        print(f"Transfer Successful! New Balance: {account_details[account_found][2]}")
                        print("-" * 50)
                        print("Transaction Details (Sender):")
                        print("-" * 33)
                        print(f"Date & Time : {ts}")
                        print(f"Transfer To : {recipient_account_no}")
                        print(f"Amount      : {transfer_amount}")
                        print(f"Status      : Transfer/Debited")
                        print(f"Balance     : {account_details[sender_account_found][2]}")
                else:
                    # If the recipient's account number is not found, inform the user
                    print("Recipient's Account Number not found.")
            else:
                # If no valid sender account is found, inform the user
                print("Invalid account number or pin.")
                
    # Option 4: View Transaction History
    elif choice == 4:
        ask_account_number = input("Enter Account Number   : ")
        ask_password = input("Enter your 6-Digit Pin : ")
    
        # Search for the account
        account_found = None
        for pancard, details in account_details.items():
            if details[3] == ask_account_number and details[4] == ask_password:
                account_found = pancard 
                break
    
        if account_found: 
            print("-" * 64)
            text4 = "Transactions History"
            padded_text4 = text4.center(64)
            print(padded_text4)
            print("-" * 64)
            print("{:<22} {:<12} {:<19} {:<12}".format("Date & Time", "Amount", "Type", "Balance"))
            print("-" * 64)
    
            # Display transaction history
            if account_found in transactions:
                for timestamp, transaction_details in transactions[account_found].items():
                    print("{:<22} {:<12} {:<19} {:<12}".format(
                        timestamp, transaction_details[0], transaction_details[1], transaction_details[2]
                    ))
            else:
                print("No transactions found for this account.")
        else:
            print("Invalid account number or pin.")
     # Option 5: Exit Program            
    elif choice == 5:
        print("*"*64)
        text5 = "Thank you for banking with WsCube Bank!"
        padded_text5 = text5.center(64)
        print(padded_text5)
        print("*"*64)
        break

**************************************************
              Welcome to WsCube Bank              
**************************************************

Menu Options: 
-------------

    Press 1 for Opening an Account
    Press 2 to View Account Details
    Press 3 to withdraw/deposit/transfer
    Press 4 to view transaction history
    Press 5 to exit the program
    
--------------------------------------------------


Enter your choice between 1-5:  6


Invalid input. Please enter a number between 1 and 5.

Menu Options: 
-------------

    Press 1 for Opening an Account
    Press 2 to View Account Details
    Press 3 to withdraw/deposit/transfer
    Press 4 to view transaction history
    Press 5 to exit the program
    
--------------------------------------------------


Enter your choice between 1-5:  1


--------------------------------------------------


...

--------------------------------------------------


Enter Account Number   :  98765432101446
Enter your 6-Digit Pin :  148543


----------------------------------------------------------------
                      Transactions History                      
----------------------------------------------------------------
Date & Time            Amount       Type                Balance     
----------------------------------------------------------------
2025-01-17 18:51:57    1000.0       Transfer/Credited   9000.0      

Menu Options: 
-------------

    Press 1 for Opening an Account
    Press 2 to View Account Details
    Press 3 to withdraw/deposit/transfer
    Press 4 to view transaction history
    Press 5 to exit the program
    
--------------------------------------------------


Enter your choice between 1-5:  5


--------------------------------------------------
****************************************************************
            Thank you for banking with WsCube Bank!             
****************************************************************
